In [43]:
import os
from random import shuffle

class A(object):
    @staticmethod
    def fileshuffle(filename, camera, basefilename='train%d.txt'):
        
        metadata = []
        with open(filename) as r:
            for line in r:
                line = line.rstrip()
                line = line.split()
                line = line[0]
                data = line.split('/')
                customer, tracklet, detid = data[-3], data[-2], data[-1]
                metadata.append((customer, tracklet, detid))
        shuffle(metadata)
        with open(basefilename % (camera,), 'w') as w:
            for m in metadata:
                _c,_t,_d = m
                w.write('./annotated/%s/%s/%s %d\n' % (_c,_t,_d, camera))
        
    @staticmethod
    def combinations(filename,camera, samples = 2, basefilename='train_combination%d.txt'):
        metadata = []
        with open(filename) as r:
            for line in r:
                line = line.rstrip()
                line = line.split()
                data = line[0].split('/')
                customer, tracklet, detid = data[-3], data[-2], data[-1]
                metadata.append((customer, tracklet, detid))
        
        train      = []
        
        for i in range(len(metadata)):
            positives = []
            negatives = []
            
            for j in range(len(metadata)-1, -1, -1):
                if i == j:
                    continue
                c1, t1, d1 = metadata[i]
                c2, t2, d2 = metadata[j]
                if c1 == c2 and len(positives) < samples:
                    positives.append(j)
                if c1 != c2 and len(negatives) < samples:
                    negatives.append(j)
                if (len(positives) == samples and len(negatives)== samples):
                    for k in range(samples):
                        train.append((i, positives[k], negatives[k], camera ))
                    break
        print('creating...')
        print(basefilename % (camera,))
        
        with open(basefilename % (camera,), 'w') as w:
            for t in train:
                w.write('%d %d %d %d \n' % t)
    
    @staticmethod
    def generate_train_data(filename, camera, samples = 2):
        A.fileshuffle(filename, camera, 'train%d.txt')
        A.combinations('train%d.txt' % (camera,), camera, samples, 'train_combination%d.txt')
        
    
    @staticmethod
    def generate_valid_data(filename, camera, samples = 2):
        A.fileshuffle(filename, camera, 'val%d.txt')
        tmp = 'tmp_val%d.txt'
        A.combinations('val%d.txt' % (camera,), camera, samples, tmp)
        A.from_train_2_val(tmp % (camera,), camera, 'val_combination%d.txt')
        
    @staticmethod
    def from_train_2_val(filename, camera, basefilename= 'val_combination%d.txt'):
        with open(basefilename % (camera,), 'w') as w:
            
            with open(filename) as r:
                for line in r:
                    line = line.rstrip()
                    line = line.split()
                    w.write('%s %s %s \n' % (line[0], line[1], camera))
                    w.write('%s %s %s \n' % (line[0], line[2], camera))
                    
                
        
    

In [40]:
A.generate_train_data('shuffled0.txt', camera=0, samples=2)

creating...
train_combination0.txt


In [44]:
A.generate_valid_data('shuffled0.txt', camera=0, samples=2)

creating...
tmp_val0.txt
